# Building a Multi-Agent Customer Support Triage System with Orkes Conductor

This notebook demonstrates the technical implementation of a multi-agent system using Orkes Conductor to handle customer support ticket triage.

## Architecture Overview

Our system consists of three specialized AI agents:
1. **Classifier Agent**: Analyzes ticket content to determine category, sentiment, and urgency
2. **Knowledge Agent**: Searches internal documentation for relevant solutions
3. **Escalation Agent**: Routes unresolved or high-priority tickets to human agents

We'll also implement a custom worker task to demonstrate how external services can integrate with the workflow.

---

## Prerequisites: Environment Setup

This notebook uses a dedicated conda environment to ensure clean dependency management. Follow the setup instructions below before running the notebook.

# Building a Multi-Agent Customer Support System with Orkes Conductor

You're about to build something powerful: a multi-agent AI system that handles customer support tickets automatically. Think of it as assembling a specialized team where each member has one job and does it exceptionally well.

## What We're Building

Three AI agents working together:
1. **Classifier Agent**: Your intake specialist—reads every ticket and figures out what's urgent, what category it falls into, and how the customer feels about it
2. **Knowledge Agent**: Your documentation expert—searches your knowledge base and suggests solutions with confidence scores
3. **Escalation Agent**: Your decision maker—routes tickets to humans when needed, auto-resolves when confident

Plus a **custom worker task** that shows you how to integrate external services (Slack, email, PagerDuty) into the workflow.

The real lesson here isn't just about AI agents. It's about **orchestration**—how you coordinate multiple specialized components into a system that's more reliable than any single agent could ever be.

---

## Before You Start: Environment Setup

This notebook uses a dedicated conda environment. Why? Because dependency conflicts are painful, and you want a clean slate. If you've ever had one project break another because of a package upgrade, you know why isolated environments matter.

Follow the setup below, then come back and run the rest of the notebook.

In [ ]:
## Step 0: Check for Conda

First things first—let's make sure you have conda installed. If not, we'll point you to the installation instructions.

In [ ]:
## Step 0.5: Create Your Environment

Run the cells below to create a conda environment named `orkes-multiagent`. This gives you an isolated Python environment with exactly the dependencies you need—nothing more, nothing less.

**Important**: After creating the environment, you need to tell Jupyter to use it:
- **Jupyter Notebook**: `Kernel` → `Change Kernel` → `orkes-multiagent`
- **JupyterLab**: Click the kernel name (top-right) → Select `orkes-multiagent`
- **VS Code**: Click the kernel selector → Choose `orkes-multiagent`

Think of this like switching which Python installation you're using—it's the same notebook, different environment.

In [ ]:
# Install ipykernel in the environment and register it as a Jupyter kernel
!conda run -n orkes-multiagent pip install ipykernel
!conda run -n orkes-multiagent python -m ipykernel install --user --name=orkes-multiagent --display-name="Python (orkes-multiagent)"

print("\n✅ Environment created successfully!")
print("\n⚠️  IMPORTANT: Now you need to switch to the 'orkes-multiagent' kernel:")
print("   1. In Jupyter: Kernel → Change Kernel → Python (orkes-multiagent)")
print("   2. In VS Code: Click kernel selector (top right) → Python (orkes-multiagent)")
print("   3. Then continue with Step 1 below")

In [ ]:
## Step 1: Install Dependencies

**Before running this**: Make sure you switched to the `orkes-multiagent` kernel (see instructions above). Otherwise, you'll install packages in the wrong environment and wonder why nothing works.

This installs the Orkes Conductor Python SDK, OpenAI client, and a few utilities.

In [ ]:
# Verify you're using the correct environment
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")

if "orkes-multiagent" in sys.executable or "orkes-multiagent" in sys.prefix:
    print("\n✅ You're using the orkes-multiagent environment!")
else:
    print("\n⚠️  WARNING: You may not be using the orkes-multiagent environment.")
    print("   Please switch kernels before continuing.")

In [ ]:
## Step 2: Connect to Orkes Conductor

Time to configure your connection to Orkes. You'll need credentials from your Orkes account (sign up for free at orkes.io if you haven't already).

Your credentials go in a `.env` file—never hardcode API keys in notebooks or scripts. This way, you can share code without sharing secrets.

In [ ]:
## Step 3: Build the Classifier Agent

Here's where it gets interesting. The Classifier Agent's job is simple: read a ticket and extract structured information—category, sentiment, urgency.

Notice what you're *not* doing: you're not calling OpenAI's API directly, not managing retries, not handling timeouts. Conductor does all of that. You just define what the agent should do.

In [ ]:
## Step 4: Build the Knowledge Agent

The Knowledge Agent builds on what the Classifier learned. See the `${classify_ticket.output.result.category}` syntax? That's data flowing from the first agent into the second.

This is multi-agent coordination in action—each agent does its job and passes the baton to the next. The workflow becomes a pipeline where intelligence accumulates.

In [ ]:
## Step 5: Create a Custom Worker Task

Here's where things get practical. LLM agents are great for reasoning, but real systems need to *do* things—send a Slack message, create a Jira ticket, trigger PagerDuty.

That's what worker tasks are for. They're custom code that runs outside Conductor but is orchestrated by it. The worker actively polls Conductor asking "got any work for me?" When a task is ready, the worker executes it and reports back.

This polling model gives you decoupling, scalability, and reliability. Your worker can live anywhere—different server, different cloud, even different language.

In [ ]:
## Step 6: Build the Escalation Agent

The Escalation Agent is your decision maker. It looks at the ticket urgency, the category, and how confident the Knowledge Agent was, then decides: escalate to a human or auto-resolve?

This is another LLM-based agent, but its job is judgment—not classification or search.

In [ ]:
## Step 7: Wire Everything Together

Now comes the satisfying part—assembling all the pieces into a complete workflow.

The `>>` operator chains tasks together. Conductor takes this definition and manages execution order, data flow, retries, and observability. You get production-ready orchestration without writing any of that plumbing yourself.

In [ ]:
## Step 8: Register the Worker (Optional)

In production, workers run as separate processes—typically containerized services that poll Conductor continuously. For this demo, we're skipping the actual worker startup (that's what `task_handler.start_processes()` would do).

The workflow will still run, but the notification task will wait for a worker that isn't there. That's okay—it demonstrates how Conductor orchestrates heterogeneous tasks (LLM agents + custom workers).

In [ ]:
## Step 9: Test It Out

Let's run this thing. Three sample tickets—login issues, a critical outage, and a billing question. Each one will flow through the entire workflow: Classifier → Knowledge → Escalation → Notification.

The workflows execute asynchronously in Orkes. You submit them here, but they run in the cloud. Check the Conductor UI to watch them in real-time.

In [ ]:
## Step 10: Monitor and Debug

Conductor gives you visibility into every workflow execution. Use this function to inspect a workflow—you'll see every task, its status, inputs, outputs, and how long it took.

When something breaks (and it will), this is how you debug. No more black boxes.

## What You Just Built

You built a production-ready multi-agent system. Here's what makes it powerful:

**Multi-Agent Orchestration**: Three specialized AI agents (Classifier, Knowledge, Escalation) working together—each doing one job exceptionally well.

**Custom Worker Integration**: A worker task that shows how to integrate external services (Slack, PagerDuty) into AI workflows.

**Production Patterns**: Built-in retry logic, error handling, state management, and observability. You don't write this yourself—Conductor provides it.

**Workflow Composition**: Easy to test, easy to modify, easy to scale. Want to add a fourth agent? Just add it to the workflow chain.

### Why This Matters

**Separation of Concerns**: Change one agent without breaking others. Deploy the new Classifier, and your workflow automatically uses it.

**Independent Scaling**: Maybe your Knowledge Agent needs GPUs. The Classifier runs fine on CPUs. With Conductor, scale each component independently.

**Observability**: When a ticket gets misrouted, you see exactly what every agent returned. No guessing.

**Flexibility**: Add duplicate detection, human-in-the-loop approval, or A/B testing different prompts—all workflow changes, not agent rewrites.

### Next Steps

1. Connect your actual LLM provider and knowledge base
2. Implement real notification channels (Slack, email, PagerDuty)
3. Add error handling and retry policies
4. Deploy workers as containerized services (see README for Docker/Kubernetes examples)
5. Set up monitoring and alerting

The future isn't bigger AI models. It's better orchestration of specialized agents. You just built the foundation for that.